In [1]:
# This code cell is to get rid of annoying tensorflow warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [4]:
df_train.head()

,id,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom,price
0,0,34291,24,1,0,47,35693,2,1,2000,0,1,8,5196,369,0,3,3436795.2
1,1,95145,60,0,1,60,34773,1,4,2000,0,1,729,4496,277,0,6,9519958.0
2,2,92661,45,1,1,62,45457,4,8,2020,1,1,7473,8953,245,1,9,9276448.1
3,3,97184,99,0,0,59,15113,1,1,2000,0,1,6424,8522,256,1,9,9725732.2
4,4,61752,100,0,0,57,64245,8,4,2018,1,0,7151,2786,863,0,7,6181908.8


In [5]:
y_train = df_train["price"]

In [6]:
X_train = df_train.drop(["id", "price"], axis=1)
X_test = df_test.drop(["id"], axis=1)

In [7]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer

In [8]:
transformer = make_column_transformer(
     (StandardScaler(), ['squareMeters', 'cityCode', 'basement', 'attic', 'hasGuestRoom']),
     (OneHotEncoder(handle_unknown='ignore'), ['numberOfRooms', 'floors', 'cityPartRange', 'numPrevOwners', 'made', 'garage']))

In [9]:
transformer.fit(X_train)

ColumnTransformer(transformers=[('standardscaler', StandardScaler(),
                                 ['squareMeters', 'cityCode', 'basement',
                                  'attic', 'hasGuestRoom']),
                                ('onehotencoder',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['numberOfRooms', 'floors', 'cityPartRange',
                                  'numPrevOwners', 'made', 'garage'])])

In [10]:
X_train = transformer.transform(X_train).toarray()
X_test = transformer.transform(X_test).toarray()

In [11]:
# from sklearn.linear_model import LinearRegression

In [13]:
from tensorflow.keras import backend as K
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [14]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [15]:
model.compile(
    loss=rmse,
    optimizer=tf.keras.optimizers.Adam(),
    metrics=[rmse]
)

In [16]:
model.fit(X_train, y_train, epochs=20)

Epoch 1/20
711/711 [==============================] - 6s 7ms/step - loss: 3332781.7500 - rmse: 3330567.2500
Epoch 2/20
711/711 [==============================] - 4s 6ms/step - loss: 585699.3125 - rmse: 585163.8125
Epoch 3/20
711/711 [==============================] - 4s 6ms/step - loss: 218710.9062 - rmse: 218511.4375
Epoch 4/20
711/711 [==============================] - 4s 6ms/step - loss: 211320.4219 - rmse: 211277.3281
Epoch 5/20
711/711 [==============================] - 4s 6ms/step - loss: 213578.8281 - rmse: 213394.5469
Epoch 6/20
711/711 [==============================] - 4s 6ms/step - loss: 209690.5156 - rmse: 209503.8594
Epoch 7/20
711/711 [==============================] - 4s 6ms/step - loss: 209464.8125 - rmse: 209275.8594
Epoch 8/20
711/711 [==============================] - 4s 6ms/step - loss: 209525.8750 - rmse: 209337.5469
Epoch 9/20
711/711 [==============================] - 4s 6ms/step - loss: 208422.7500 - rmse: 208237.5000
Epoch 10/20
711/711 [=======================

In [23]:
ans = np.array(model.predict(X_test))

474/474 [==============================] - 1s 3ms/step


In [30]:
ans = np.squeeze(ans)

In [31]:
ans.shape

(15154,)

In [32]:
df_ans = pd.DataFrame({"id": df_test["id"], "price": ans})

In [33]:
df_ans.to_csv("outputs/ans4.csv", index=False)